In [2]:
from transformers import pipeline
import pandas as pd

text = """After 6 months my daughter was complaining how the left airpod volume was considerably lower than right. That bad that she reverted back to the wired ones.

Amazon were great in organising replacement. Issues arrised when I received an email suggesting I sent back the wrong airpods, serial numbers not matching. I contacted Apple, and they advised the case and airpods serial numbers should all match. They did advise possibly my daughter mixed up her case with a friends.

This got me thinking, I checked the serial numbers of the replacement, the only serial number that matched that of the box was the case. So, either non genuine, or refurbished.

I suggest purchasing from JBHIFI, Officeworks or the Goodguys. I ended up purchasing from the Goodguys and they price matched Officeworks special $189.00.

If you do go down the path of purchasing from Amazon store, I encourage you to video the unboxing and serial numbers for evidence.
"""

## Sentiment Analysis (Classifier)

In [4]:
# %%
classifier = pipeline('text-classification')

outputs = classifier(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,label,score
0,NEGATIVE,0.999186


## Name Entity Recognition

In [7]:
model = "dbmdz/bert-large-cased-finetuned-conll03-english"
# %% Named Entity Recognition
ner_tagger = pipeline(task = "ner", model = model, aggregation_strategy = "simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)


,entity_group,score,word,start,end
0,ORG,0.999340,Amazon,157,163
1,ORG,0.998753,Apple,325,330
2,ORG,0.996561,JBHIFI,679,685
3,ORG,0.999495,Officeworks,687,698
4,ORG,0.993956,Goodguys,706,714
5,ORG,0.991882,Goodguys,747,755
6,ORG,0.998596,Officeworks,779,790
7,ORG,0.985852,Amazon,855,861


## Question and Answer

In [15]:
# %% Q&A
model = "deepset/roberta-base-squad2"
reader = pipeline(task = "question-answering", model = model)
question = "What is the problem?"
outputs = reader(question = question, context = text)
print(outputs)


{'score': 0.11276600509881973, 'start': 51, 'end': 103, 'answer': 'left airpod volume was considerably lower than right'}


## Summarisation

In [13]:
# %% Summarisation
summariser = pipeline(task = "summarization")
outputs = summariser(text, max_length = 60, clean_up_tokenization_spaces = True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 After 6 months my daughter was complaining how the left airpod volume was considerably lower than right. That bad that she reverted back to the wired ones. Amazon were great in organising replacement. Apple advised the case and airpods serial numbers should all match. The only serial number that matched


## Translation

In [3]:
# %% Translation
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline

mode_name = 'Helsinki-NLP/opus-mt-en-zh'
model = AutoModelWithLMHead.from_pretrained(mode_name)
tokenizer = AutoTokenizer.from_pretrained(mode_name)

translation = pipeline(task = "translation_en_to_zh", model = model, tokenizer = tokenizer)
translation(text)


/mnt/c/dev/machine_learning_deep_learning/.env/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/mnt/c/dev/machine_learning_deep_learning/.env/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': '6个月后,我的女儿抱怨说左空投的量大大低于右。 糟糕的是, 她又回到了有线飞机上。 亚马逊在组织替换方面非常出色。 当我收到一封电子邮件, 暗示我寄回错误的空投, 序列号不匹配时, 问题就大为不妙了。 我联系了苹果, 他们告诉了这个案例, 空投的序列号应该都匹配了。 他们建议我的女儿可能把她的案件和朋友混在一起。 这让我想到, 我检查了替换的序列号, 替换的序列号是唯一与盒子相符的序列号。 因此, 要么是不真实的, 要么是翻新的。 我建议从JBHIFI、 Officeworks 或 Goodguys 购买。 我最后从Goodguys购买了价格, 并且他们的价格与办公室的特价189. 00 。 如果你从亚马逊商店购买的话, 我鼓励你录下解字和序列号作为证据。'}]

## Text Generation

In [7]:
# %% Text Generation
generator = pipeline("text-generation", model = "gpt2")
response = "Dear customer, sorry to hear about your issue."
prompt = text + "\n\nCustomer Service Response:\n" + response
outputs= generator(prompt, max_length = 400)
print(outputs[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


After 6 months my daughter was complaining how the left airpod volume was considerably lower than right. That bad that she reverted back to the wired ones.

Amazon were great in organising replacement. Issues arrised when I received an email suggesting I sent back the wrong airpods, serial numbers not matching. I contacted Apple, and they advised the case and airpods serial numbers should all match. They did advise possibly my daughter mixed up her case with a friends.

This got me thinking, I checked the serial numbers of the replacement, the only serial number that matched that of the box was the case. So, either non genuine, or refurbished.

I suggest purchasing from JBHIFI, Officeworks or the Goodguys. I ended up purchasing from the Goodguys and they price matched Officeworks special $189.00.

If you do go down the path of purchasing from Amazon store, I encourage you to video the unboxing and serial numbers for evidence.


Customer Service Response:
Dear customer, sorry to hear ab

In [25]:
translation = pipeline(task = "translation_en_to_zh")

ValueError: The task does not provide any default models for options ('en', 'zh')